# Step I - Import Libraries and get the list of all equity derivatives

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import pandas_datareader.data as pdr
import talib as ta
import math

C:\Users\akshay\anaconda3\envs\algo\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
stocks = pd.read_csv('NSE_derivatives_stocks.csv',encoding= 'unicode_escape')

In [3]:
ticker = stocks['ticker']

In [4]:
ticker = [item+'.NS' for item in ticker]

# Step II - Fetch data from Yahoo Finance API

In [5]:
start = dt.datetime.today() - dt.timedelta(360)
end = dt.datetime.today()

In [6]:
data = pdr.get_data_yahoo(ticker,start,end)

In [65]:
data_close = data['Adj Close']
price = data_close.copy()

In [66]:
price_pct = price.pct_change(1)

# Step III - Apply the scans

# 1. Price Change > 3%

In [67]:
scanned_stocks = []

In [68]:
for i in ticker:
    if price_pct[i][-1] < -0.03:
        scanned_stocks.append(i)

# 2. Price > 200 SMA and RSI(2) > 50

In [69]:
final_stocks = []

In [70]:
for i in scanned_stocks:
    if (ta.SMA(price[i],200)[-1] > price[i][-1]) & (ta.RSI(price[i],2)[-1] < 50):
        final_stocks.append(i)

In [71]:
cp_stocks = price[final_stocks].iloc[-1].round(1)

# Step IV - Find the Entry, Stop loss and Target price

In [72]:
entry_price = (cp_stocks*0.99).round(1)
entry_price = pd.DataFrame(entry_price)
sl_price = (entry_price* 0.97).round(1)
tgt_price = (entry_price* 1.05).round(1)

In [73]:
order_df = pd.concat([entry_price,sl_price,tgt_price],axis =1,ignore_index = True)

In [74]:
order_df.rename(columns = {0:'Entry_Price', 1:'Stop_Loss', 2:'Target'},inplace = True)

In [75]:
order_df

,Entry_Price,Stop_Loss,Target
Symbols,,,


# Step V - Position Sizing

In [76]:
trading_capital = 10000        #Enter the trading capital

In [77]:
try:
    max_cap = trading_capital/order_df.shape[0]
except:
    print('There are no stocks for this strategy today')

There are no stocks for this strategy today


In [78]:
order_df['Qty'] = np.floor(max_cap/order_df['Entry_Price'])

In [79]:
order_df['Max_Loss'] = order_df['Qty']*(order_df['Entry_Price'] - order_df['Stop_Loss'])

In [80]:
order_df['Max_Profit'] = order_df['Qty']*(order_df['Target'] - order_df['Entry_Price'])

In [81]:
order_df.loc['Total_Max_Loss','Max_Loss'] = order_df['Max_Loss'].sum()

In [82]:
order_df.loc['Total_Max_Profit','Max_Profit'] = order_df['Max_Profit'].sum()

In [83]:
order_df = order_df.replace(np.nan,'',regex = True)

In [84]:
order_df

,Entry_Price,Stop_Loss,Target,Qty,Max_Loss,Max_Profit
Symbols,,,,,,
Total_Max_Loss,,,,,0,
Total_Max_Profit,,,,,,0


In [85]:
order_df.to_csv('trade-data/2020-05-29.csv')